In [1]:
import pandas as pd
import sys
import os
import datetime

In [ ]:
operation = "rent"
city = "lisbon"

read_path = f"../data/idealista/raw/{operation}/{city}"
save_path = f"../data/idealista/unioned/{operation}/{city}"

dataframes = []

for file in os.listdir(read_path):
    file_date = file[:10]
    if file.endswith(".csv") and file_date.st:
        dataframes.append(pd.read_csv(f"{read_path}/{file}", index_col='propertyCode'))
        dataframes[-1]["snapshotDate"] = datetime.datetime.strptime(file_date, "%Y-%m-%d").date()

In [2]:
# Get the absolute path of the scripts directory
root_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))

# Add the scripts directory to Python's module search path
sys.path.append(root_dir)

In [3]:
from scripts.preprocessors import IdealistaDataLoader

dataframes = []

data_loader = IdealistaDataLoader(
    read_path=f"../data/idealista",
    city="lisbon",
    operation="rent",
    date_or_unioned="unioned",
    include_geodata=True,
    include_openai=True,
    index_col="propertyCode",
    directory="cleaned"
)

dataframes.append(data_loader.load_data())

data_loader = IdealistaDataLoader(
    read_path=f"../data/idealista",
    city="lisbon",
    operation="rent",
    date_or_unioned="2025-03-04",
    include_geodata=True,
    include_openai=True,
    index_col="propertyCode",
    directory="cleaned"
)

dataframes.append(data_loader.load_data())

In [4]:
# Combine the DataFrames
df = pd.concat(dataframes)

# Keep only the last occurrence of each index
df = df.loc[~df.index.duplicated(keep="last")]

# Keep only the listings that have unique values for the following columns
df = df.loc[
    ~df[
        [
            "floor",
            "propertyType",
            "size",
            "rooms",
            "bathrooms",
            "address",
            "description",
        ]
    ].duplicated(keep="last")
]



In [5]:
df.index.is_unique

True

In [ ]:
operation = "rent"
city = "lisbon"

save_path = f"../data/idealista/cleaned/{operation}/{city}"

save_path += f"/unioned-{city}-listings-for-{operation}-with-geodata-openai.csv"

In [9]:
save_path

'../data/idealista/unioned/rent/lisbon/unioned-lisbon-listings-for-rent-with-geodata-openai.csv'

In [ ]:
# Save the unioned DataFrame to a new CSV file
df.to_csv(save_path, index_label="propertyCode")

In [ ]:
df